This notebooks allows you to play against a machine (either Stockfish or a Transformer) using the python-chess library.
It uses the UCI protocol to communicate with the engine, and you can set the engine's options, choose board side and make moves.

Let's setup the environment.

In [ ]:
import yaml
import chess
from chess import Board
from chess.pgn import Game
import ipywidgets as widgets
from IPython.display import display, SVG, Markdown
from utils.utils import load_engine, engine_move, human_move, get_game_pgn, LICHESS_LEVELS

config_path = '../config.yaml'

try:
    with open(config_path) as f:
        config = yaml.safe_load(f)
except FileNotFoundError:
    raise FileNotFoundError(f"Configuration file not found at '{config_path}'.")

Select the desired strength level, your name and the side you want to play as (white or black).

In [ ]:
stockfish_path = config['stockfish_path']
YOURNAME = "Jacopo"
LEVEL = 1 # 1-8
SIDE = "w" # for white, or "b" for black

Now you can play! Make sure you enter valid moves in UCI format (e.g., e2e4, g1f3)!

In [ ]:
try:
    engine = load_engine(stockfish_path)
    engine.configure({"Skill Level": LICHESS_LEVELS[LEVEL]['SKILL']})
except FileNotFoundError:
    print(f"ERROR: Stockfish engine not found at '{stockfish_path}. Please ensure the path is correct and the file is executable.")
except Exception as e:
    print(f"An error occurred initializing the Stockfish engine: {e}")

# Initialize the board
board = Board()

if SIDE == "b":
    board = engine_move(board, engine, engine_config=LICHESS_LEVELS[LEVEL])

# Create the board display
board_display = display(SVG(chess.svg.board(board=board, size=350)), display_id=True)

def update_board(board, move_uci_string):
    try:
        # --- Player's Move ---
        board = human_move(board=board, move=move_uci_string)
        if board is None:
            return

        # --- Check Game Over After Player's Move ---
        if check_game_over(board):
            return

        # --- Engine's Move ---
        board = engine_move(board, engine, engine_level=LEVEL)
        board_display.update(SVG(chess.svg.board(board=board, size=350)))

    except Exception as e:
        display(f"An unexpected error occurred during move processing: {e}")

def handle_submit(_):
    if move_input.value.lower() == "r":
        display(f"You resigned. Game Over!")
        res = "0-1" if SIDE == "w" else "1-0"
        w_name = YOURNAME if SIDE == "w" else "Stockfish"
        b_name = "Stockfish" if SIDE == "w" else YOURNAME
        display(Markdown(f"Game pgn:\n```{get_game_pgn(board=board, w_name=w_name, b_name=b_name, result=res)}```"))
        return

    elif move_input.value.lower() == "d":
        display(f"You offered a draw. Game Over!")
        res = "1/2-1/2"
        w_name = YOURNAME if SIDE == "w" else "Stockfish"
        b_name = "Stockfish" if SIDE == "w" else YOURNAME
        display(Markdown(f"Game pgn:\n```{get_game_pgn(board=board, w_name=w_name, b_name=b_name, result=res)}```"))
        # Disable input and button
        move_input.disabled = True
        submit_button.disabled = True
        return 
    
    update_board(board, move_input.value)
    move_input.value = ""

def check_game_over(board):
    if board.is_game_over():
        display("Game Over!")
        w_name = YOURNAME if SIDE == "w" else "Stockfish"
        b_name = "Stockfish" if SIDE == "w" else YOURNAME
        display(Markdown(f"Game pgn:\n```{get_game_pgn(board=board, w_name=w_name, b_name=b_name)}```"))

        # Disable input and button
        move_input.disabled = True
        submit_button.disabled = True
        return True
    return False

move_input = widgets.Text(placeholder="UCI move, 'r' to resign, 'd' to offer a draw",
                          description="Your move:",
                          layout=widgets.Layout(width="350px", height="50px"))
submit_button = widgets.Button(description="Submit choice")
submit_button.on_click(handle_submit)

display(widgets.HBox([move_input, submit_button]))